In [1]:
import tensorflow as tf
import numpy as np
import os
import string
from captcha.image import ImageCaptcha

In [2]:
charset = string.digits + string.ascii_uppercase#string.ascii_letters
charset = np.array(list(charset))
height, width, n_len, n_classes = 64, 128, 4, len(charset)

batch_size = 64

output_dir = './tfrecords/captcha_py'

In [3]:
def capt_gen(batch_size=batch_size, n_len=n_len):
    gen = ImageCaptcha(height=height, width=width)
    
    while True:
        x, y = [], []
        for i in range(batch_size):
            chars_idx = [np.random.choice(range(n_classes)) for _ in range(n_len)]
            x.append(np.array(gen.generate_image(charset[chars_idx])))
            y.append(chars_idx)
        
        x, y = np.array(x), np.array(y)
        if (x.shape != (batch_size, height, width, 3)) or (y.shape != (batch_size, n_len)):
            continue
        yield x, y

        
def capt_decode(y):
    return ''.join([charset[i] for i in y])


if __name__ == '__main__':
    gen = capt_gen()
    x, y = next(gen)
    print(x.shape, y.shape)

    a, b = next(gen)
    print(np.sum(a != x), np.sum(b != y))
    
    print(capt_decode(next(gen)[1][0]))

(64, 64, 128, 3) (64, 4)
1529580 251
NOBN


In [4]:
def _tfrecords_gen():

    def _capt_gen_single(batch_size=batch_size, n_len=n_len):
        gen = ImageCaptcha(height=height, width=width)

        while True:
            chars_idx = [np.random.choice(range(n_classes)) for _ in range(n_len)]
            x = gen.generate(charset[chars_idx]).getvalue()
            y = np.array(chars_idx).astype(np.uint8)

            yield x, y

    gen = _capt_gen_single()    

    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    
    def _bytes_feature(value):
        return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))
    
    for i in range(40):
        path = os.path.join(output_dir, 'captcha_%d.tfrecords' % i)
        
        with tf.python_io.TFRecordWriter(path) as writer:        
            for _ in range(10000):
                image, label = next(gen)
                label_raw = label.tobytes()

                example = tf.train.Example(
                    features=tf.train.Features(
                        feature={
                            'image': _bytes_feature(image),
                            'label': _bytes_feature(label_raw)
                        }
                    )
                )
                writer.write(example.SerializeToString())

In [5]:
if __name__ == '__main__':
    ensure = input('Write tfrecords ? (y/n): ')
    if ensure == 'y':
        _tfrecords_gen()

Write tfrecords ? (y/n): n
